In [29]:
from rank_bm25 import BM25Okapi
import pandas as pd

df = pd.read_csv("data/parsed_docs.csv")

In [27]:
import pytrec_eval
import json

qrel_file = "data/qrel_train.txt"

qrel_json = {}
with open(qrel_file, "r") as f:
    for line in f.readlines():
        line_split = line.split("\t")
        query_id = line_split[0]
        doc = line_split[2]
        rel = int(line_split[3].strip())

        if query_id in qrel_json:
            qrel_json[query_id][doc]=rel
        else:
            qrel_json[query_id]={doc:rel}


evaluator = pytrec_eval.RelevanceEvaluator(qrel_json, {'map', 'ndcg',"recall", "Rprec"})
print(json.dumps(evaluator.evaluate(qrel_json), indent=1))

{
 "16": {
  "map": 1.0,
  "Rprec": 1.0,
  "recall_5": 0.14705882352941177,
  "recall_10": 0.29411764705882354,
  "recall_15": 0.4411764705882353,
  "recall_20": 0.5882352941176471,
  "recall_30": 0.8823529411764706,
  "recall_100": 1.0,
  "recall_200": 1.0,
  "recall_500": 1.0,
  "recall_1000": 1.0,
  "ndcg": 1.0
 },
 "17": {
  "map": 1.0,
  "Rprec": 1.0,
  "recall_5": 0.45454545454545453,
  "recall_10": 0.9090909090909091,
  "recall_15": 1.0,
  "recall_20": 1.0,
  "recall_30": 1.0,
  "recall_100": 1.0,
  "recall_200": 1.0,
  "recall_500": 1.0,
  "recall_1000": 1.0,
  "ndcg": 1.0
 },
 "18": {
  "map": 1.0,
  "Rprec": 1.0,
  "recall_5": 0.19230769230769232,
  "recall_10": 0.38461538461538464,
  "recall_15": 0.5769230769230769,
  "recall_20": 0.7692307692307693,
  "recall_30": 1.0,
  "recall_100": 1.0,
  "recall_200": 1.0,
  "recall_500": 1.0,
  "recall_1000": 1.0,
  "ndcg": 1.0
 },
 "19": {
  "map": 1.0,
  "Rprec": 1.0,
  "recall_5": 0.625,
  "recall_10": 1.0,
  "recall_15": 1.0,
  "re

In [2]:
df

,docno,text
0,1050324_business_story_4530765.utf8,The Telegraph - Calcutta : Business Tighter ...
1,1050907_business_story_5207471.utf8,The Telegraph - Calcutta : Business Global S...
2,1050811_business_story_5100703.utf8,The Telegraph - Calcutta : Business RBI eagl...
3,1050610_business_story_4849980.utf8,The Telegraph - Calcutta : Business Posco de...
4,1051226_business_story_5645448.utf8,The Telegraph - Calcutta : Business LOOSE CH...
...,...,...
392534,en.3.403.491.2007.11.7,"SINGAPORE, Nov 7 (bdnews24.com/Reuters) - Oil ..."
392535,en.3.403.360.2007.11.4,"CHICAGO, Nov 4 (bdnews24.com/Reuters) - An eff..."
392536,en.3.403.430.2007.11.5,"PARIS, Nov 5 (bdnews24.com/Reuters) - Airbus p..."
392537,en.3.403.407.2007.11.6,"Pabna, Nov 5 (bdnews24.com) Awami League lead..."


In [7]:
import swifter
corpus = df.text.tolist()
tokenized_docs = df["text"].swifter.apply(lambda x: [k.lower() for k in x.split()])

Pandas Apply: 100%|██████████| 392539/392539 [00:34<00:00, 11454.50it/s]


In [8]:
bm25_idx = BM25Okapi(tokenized_docs)

In [33]:
r = bm25_idx.get_scores([tok.lower() for tok in "Telecom minister A. Raja resignation".split()])

In [37]:
import numpy as np
sorted_scores_idx = np.argsort(r)[::-1][:20]

In [38]:
run = {"16":{}}
query = "16"
for idx in sorted_scores_idx:
    score = r[idx]
    docno = df.iloc[idx].docno
    text = df.iloc[idx].text
    run["16"][docno]=score

In [39]:
print(json.dumps(evaluator.evaluate(run), indent=1))

{
 "16": {
  "map": 0.07434640522875817,
  "ndcg": 0.22922431489191336
 }
}


In [13]:
txt = "tes.v.s"
txt_id = int("".join([str(ord(chr)) for chr in txt]))

In [35]:
import spacy
import swifter

nlp = spacy.load('en')

df["processed_text"] = df["text"].swifter.apply(lambda x: [tok.lemma_.lower() for tok in nlp(x)])


Pandas Apply: 100%|██████████| 392539/392539 [5:04:28<00:00, 21.49it/s]


,docno,text,processed_text
0,1050324_business_story_4530765.utf8,The Telegraph - Calcutta : Business Tighter ...,"[the, telegraph, -, calcutta, :, business, ,..."
1,1050907_business_story_5207471.utf8,The Telegraph - Calcutta : Business Global S...,"[the, telegraph, -, calcutta, :, business, ,..."
2,1050811_business_story_5100703.utf8,The Telegraph - Calcutta : Business RBI eagl...,"[the, telegraph, -, calcutta, :, business, ,..."
3,1050610_business_story_4849980.utf8,The Telegraph - Calcutta : Business Posco de...,"[the, telegraph, -, calcutta, :, business, ,..."
4,1051226_business_story_5645448.utf8,The Telegraph - Calcutta : Business LOOSE CH...,"[the, telegraph, -, calcutta, :, business, ,..."
...,...,...,...
392534,en.3.403.491.2007.11.7,"SINGAPORE, Nov 7 (bdnews24.com/Reuters) - Oil ...","[singapore, ,, nov, 7, (, bdnews24.com/reuter,..."
392535,en.3.403.360.2007.11.4,"CHICAGO, Nov 4 (bdnews24.com/Reuters) - An eff...","[chicago, ,, nov, 4, (, bdnews24.com/reuter, )..."
392536,en.3.403.430.2007.11.5,"PARIS, Nov 5 (bdnews24.com/Reuters) - Airbus p...","[paris, ,, nov, 5, (, bdnews24.com/reuters, ),..."
392537,en.3.403.407.2007.11.6,"Pabna, Nov 5 (bdnews24.com) Awami League lead...","[pabna, ,, nov, 5, (, bdnews24.com, ), , awam..."


In [37]:
import pickle
import compress_pickle

In [38]:
pickle.dump(df, open("parsed_docs_lemmas.pkl", "wb"))

In [39]:
compress_pickle.dump(df, "parsed_docs_lemmas.pkl.gz")

In [41]:
df2 = compress_pickle.load( "parsed_docs_lemmas.pkl.gz")